In [12]:
# import PyTorch
import torch

# standard DS stack
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
# embed static images in the ipynb
%matplotlib inline 

# neural network package
import torch.nn as nn 
import torch.nn.functional as F

# computer vision
import torchvision
from torchvision import transforms
from PIL import Image

# dataset loading
from torch.utils.data import Dataset, DataLoader

# convenient package for plotting loss functions
# from livelossplot import PlotLosses

import copy
import importlib.util # to run outside module
from sklearn.model_selection import train_test_split

In [13]:
# sns.get_dataset_names()

titanic_data = sns.load_dataset('titanic')

def one_hot_encode(X, cat_features):
    """Converts an unstructued 2-D array consisting of categorical values
    into a 2-D array consisting of one-hot-encoded vectors.

    
        X (pd.DataFrame): 
        cat_features (list): categorical feature names
    """ 
    
    X_cat = X[cat_features]
    for cat in cat_features[:]:
        X = X.drop(cat, axis=1)

    # Replace the nonnumerical columns with one-hot encoded ones.
    for name in cat_features[:]:
        hot_one = pd.get_dummies(X_cat[name], prefix=name)
        X = pd.concat([X, hot_one.set_index(X.index)], axis=1)
    return X

def preprocess_titanic(df=titanic_data):
    df['sex'] = (df['sex'].values.astype(str) == 'male').astype(int)
    df['alone'] = df['alone'].values.astype(int)
    df.drop(['deck', 'alive'], axis=1,  inplace=True)
    df = one_hot_encode(df, 
        ['who', 'embark_town', 'embarked', 'adult_male', 'class'])
    df.dropna(inplace=True)
    df = df.astype(float)
    A = df.values
    X, Y = A[:, 1:], A[:, 0].reshape(-1,1) 
    return X, Y
    
X, Y = preprocess_titanic()

# Perform train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=7)

In [16]:
# GPU for cloud training
if torch.cuda.is_available(): 
    device = torch.device("cuda") # device = GPU
else:
    device = torch.device("cpu") # device = CPU
    
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(n_features, H_0)
        self.fc2 = nn.Linear(H_0, H_1)
        self.fc3 = nn.Linear(H_1, D_out)
        
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, z):
        z = F.leaky_relu(self.fc1(z))
        z = F.leaky_relu(self.fc2(z)) 
        z = F.leaky_relu(self.fc3(z))
        return z

    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(n_features, H_0)
        self.fc2 = nn.Linear(H_0, H_1)
        self.fc3 = nn.Linear(H_1, D_out)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
#          x = self.dropout()
        x = F.leaky_relu(self.fc2(x)) 
        x = F.leaky_relu(self.fc3(x))
        return x

class Toy_Dataset(Dataset): # inherit from torch's Dataset class.
    def __init__(self, train):
        # data loading
        if train == True:
            self.X = torch.from_numpy(X_train.astype(np.float32))
            self.Y = torch.from_numpy(Y_train.astype(np.float32))
        else:
            self.X = torch.from_numpy(X_test.astype(np.float32))
            self.Y = torch.from_numpy(Y_test.astype(np.float32))

        if self.X.shape[0] == self.Y.shape[0]:
            self.n_samples = self.X.shape[0]
        else:
            raise ValueError("Shape mismatch")
        
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    
    def __len__(self):
        return self.n_samples
        # len(dataset)

# Initialize constants
n_features = X_train.shape[1]
BATCH_SIZE, D_in, D_out = 15, n_features, 1
H_0, H_1 = int(0.7*n_features), int(0.4*n_features)

# Set DataLoaders    
train_set = Toy_Dataset(train=True)
test_set = Toy_Dataset(train=False)
train_dl = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

# Initialize networks
G = Generator()
D = Discriminator()

loss_fn = nn.BCEWithLogitsLoss()

def check_models():
    x = torch.from_numpy(X_train[0]).float() 
    print( G(x) )
    print( D(x) )
    
check_models()

tensor([-0.0076], grad_fn=<LeakyReluBackward0>)
tensor([-0.0026], grad_fn=<LeakyReluBackward0>)


In [8]:
nn.Sequential?

Init signature: nn.Sequential(*args)
Docstring:     
A sequential container.
Modules will be added to it in the order they are passed in the constructor.
Alternatively, an ordered dict of modules can also be passed in.

To make it easier to understand, here is a small example::

    # Example of using Sequential
    model = nn.Sequential(
              nn.Conv2d(1,20,5),
              nn.ReLU(),
              nn.Conv2d(20,64,5),
              nn.ReLU()
            )

    # Example of using Sequential with OrderedDict
    model = nn.Sequential(OrderedDict([
              ('conv1', nn.Conv2d(1,20,5)),
              ('relu1', nn.ReLU()),
              ('conv2', nn.Conv2d(20,64,5)),
              ('relu2', nn.ReLU())
            ]))
Init docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
File:           c:\users\uniqu\anaconda3\envs\ds_env\lib\site-packages\torch\nn\modules\container.py
Type:           type
Subclasses:     ConvReLU2d, ConvReLU3d, Linea

In [9]:
nn.Tanh?

Init signature: nn.Tanh()
Docstring:     
Applies the element-wise function:

.. math::
    \text{Tanh}(x) = \tanh(x) = \frac{\exp(x) - \exp(-x)} {\exp(x) + \exp(-x)}

Shape:
    - Input: :math:`(N, *)` where `*` means, any number of additional
      dimensions
    - Output: :math:`(N, *)`, same shape as the input

.. image:: scripts/activation_images/Tanh.png

Examples::

    >>> m = nn.Tanh()
    >>> input = torch.randn(2)
    >>> output = m(input)
Init docstring: Initializes internal Module state, shared by both nn.Module and ScriptModule.
File:           c:\users\uniqu\anaconda3\envs\ds_env\lib\site-packages\torch\nn\modules\activation.py
Type:           type
Subclasses:     


In [4]:
def train():
    pass

## References

Conceptual References:
- [*GANS*, Google Developers](https://developers.google.com/machine-learning/gan/gan_structure)
- [Ian Goodfellow on Lex Fridman podcast](https://www.youtube.com/watch?v=Z6rxFNMGdn0&t=2826s&ab_channel=LexFridman)
- [Goodfellow, I., Pouget-Abadie, J., Mirza, M., Xu, B., Warde-Farley, D., Ozair, S., ... & Bengio, Y. (2014). Generative adversarial nets. In *Advances in neural information processing systems* (pp. 2672-2680).](https://arxiv.org/pdf/1406.2661.pdf)
- [Salimans, T., Goodfellow, I., Zaremba, W., Cheung, V., Radford, A., & Chen, X. (2016). Improved techniques for training gans. In *Advances in neural information processing systems* (pp. 2234-2242).](https://arxiv.org/pdf/1606.03498.pdf)

Implementation References:
- [eriklindernoren/PyTorch-GAN](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py)
  - [list of successful GAN architectures](https://github.com/eriklindernoren/PyTorch-GAN#gan)
- [fast ai, free GPU w/ Google Colab guide](https://www.kdnuggets.com/2018/02/fast-ai-lesson-1-google-colab-free-gpu.html)
- [PyTorch GAN, github.com/devnag](https://github.com/devnag/pytorch-generative-adversarial-networks/blob/master/gan_pytorch.py)
- [Brownlee, Jason (2020). How to Code the GAN Training Algorithm. *machinelearningmastery.com*](https://machinelearningmastery.com/how-to-code-the-generative-adversarial-network-training-algorithm-and-loss-functions/)